# Prediction of Health Measures Based on Location

**Data Scientist:** Elina Rankova

[IMAGE PLACEHOLDER]



## 1. Business Problem and Understanding

In [87]:
# Import libraries

import pandas as pd
import numpy as np
import random
np.random.seed(42)
random.seed(42)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## 2. Data Understanding

In [88]:
# Load datasets

pl1 = pd.read_csv('Data/PLACES_2017-2018.csv')
pl2 = pd.read_csv('Data/PLACES_2018-2019.csv')
pl3 = pd.read_csv('Data/PLACES_2019-2020.csv')
pl4 = pd.read_csv('Data/PLACES_2020-2021.csv')
sdoh = pd.read_csv('Data/SDOH_2017-2021.csv')

We may want to check on the `StateAbbr` and `StateDesc` since it's showing the country in the preview. We also want to adjust the LocationName to either exclude 'county' or include 'county' to make sure both of the datasets are aligned in nomenclature.

In [103]:
# Concat all PLACES datasets
pl_all = pd.concat([pl1, pl2, pl3, pl4])
display(pl_all.tail())

sdoh.head()

,Year,StateAbbr,StateDesc,LocationName,DataSource,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,...,High_Confidence_Limit,TotalPopulation,Latitude,CategoryID,MeasureId,DataValueTypeID,Short_Question_Text,Geolocatioin,LocationID,Geolocation
228765,2021,TX,Texas,Brown,BRFSS,Health Outcomes,High cholesterol among adults aged >=18 years ...,%,Crude prevalence,40.4,...,45.5,38192,NaN,HLTHOUT,HIGHCHOL,CrdPrv,High Cholesterol,NaN,48049.0,POINT (-98.998456 31.7641372)
228766,2021,TX,Texas,Grayson,BRFSS,Health Outcomes,Chronic kidney disease among adults aged >=18 ...,%,Age-adjusted prevalence,2.8,...,3.2,139336,NaN,HLTHOUT,KIDNEY,AgeAdjPrv,Chronic Kidney Disease,NaN,48181.0,POINT (-96.675693 33.624524)
228767,2021,SC,South Carolina,Richland,BRFSS,Health Risk Behaviors,No leisure-time physical activity among adults...,%,Crude prevalence,23.9,...,27.5,418307,NaN,RISKBEH,LPA,CrdPrv,Physical Inactivity,NaN,45079.0,POINT (-80.8980371 34.0290952)
228768,2021,WA,Washington,Franklin,BRFSS,Disability,Independent living disability among adults age...,%,Crude prevalence,8.4,...,9.8,98268,NaN,DISABLT,INDEPLIVE,CrdPrv,Independent Living Disability,NaN,53021.0,POINT (-118.9038912 46.5375019)
228769,2020,WI,Wisconsin,La Crosse,BRFSS,Health Outcomes,All teeth lost among adults aged >=65 years,%,Crude prevalence,8.1,...,11.8,120433,NaN,HLTHOUT,TEETHLOST,CrdPrv,All Teeth Lost,NaN,55063.0,POINT (-91.1117584 43.9082222)


,Year,StateAbbr,StateDesc,LocationName,DataSource,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,MOE,TotalPopulation,LocationID,CategoryID,MeasureID,DataValueTypeID,Short_Question_Text,Geolocation
0,2017-2021,AL,Alabama,Coosa County,5-year ACS,SDOH,Housing cost burden among households,%,Percentage,12.8,4.0,10442,1037,SDOH,HCOST,Percent,Housing cost burden,POINT (-86.2434818 32.9314453)
1,2017-2021,AL,Alabama,Lamar County,5-year ACS,SDOH,Persons living below 150% of the poverty level,%,Percentage,27.4,4.0,13929,1075,SDOH,POV150,Percent,Poverty,POINT (-88.0874309 33.7870852)
2,2017-2021,AK,Alaska,Aleutians East Borough,5-year ACS,SDOH,Crowding among housing units,%,Percentage,4.6,2.3,3409,2013,SDOH,CROWD,Percent,Crowding,POINT (-161.9974772 55.2450437)
3,2017-2021,AR,Arkansas,Dallas County,5-year ACS,SDOH,Crowding among housing units,%,Percentage,2.3,2.2,6645,5039,SDOH,CROWD,Percent,Crowding,POINT (-92.6539989 33.9678233)
4,2017-2021,AR,Arkansas,Franklin County,5-year ACS,SDOH,Crowding among housing units,%,Percentage,2.6,1.8,17159,5047,SDOH,CROWD,Percent,Crowding,POINT (-93.8876647 35.5085579)


We will also have to deal with the differences in the `Year` column since it is an object in the SDOH data and an integer in the PLACES data.

In [105]:
print(sdoh.info())
print(pl_all.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28287 entries, 0 to 28286
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year                 28287 non-null  object 
 1   StateAbbr            28287 non-null  object 
 2   StateDesc            28287 non-null  object 
 3   LocationName         28278 non-null  object 
 4   DataSource           28287 non-null  object 
 5   Category             28287 non-null  object 
 6   Measure              28287 non-null  object 
 7   Data_Value_Unit      28287 non-null  object 
 8   Data_Value_Type      28287 non-null  object 
 9   Data_Value           28287 non-null  float64
 10  MOE                  28285 non-null  float64
 11  TotalPopulation      28287 non-null  int64  
 12  LocationID           28287 non-null  int64  
 13  CategoryID           28287 non-null  object 
 14  MeasureID            28287 non-null  object 
 15  DataValueTypeID      28287 non-null 

In [ ]:
# Merge all datasets together
df_all = pd.concat([pl1, pl2, pl3, pl4, sdoh])

Right away we can tell that there are a few columns that are missing too many values and will need to be dropped.
- `Data_Value_Footnote_Symbol`
- `Data_Value_Footnote`
- `Latitude`
- `MOE`

`Geolocatioin` and `Geolocation` are the same feature when checking the source data websites only the 2020 release has a `Geolocatioin` column while the rest have `Geolocation`. `MeasureID` and MeasureId` have the same issue it seems.

There are object type columns that will need to be transformed.

In [90]:
print(df_all.info(),'\n')

# Drop unneeded columns
df_all.drop(columns=['Data_Value_Footnote_Symbol','Data_Value_Footnote','Latitude','MOE'],inplace=True)

<class 'pandas.core.frame.DataFrame'>
Index: 809177 entries, 0 to 28286
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Year                        809177 non-null  object 
 1   StateAbbr                   809177 non-null  object 
 2   StateDesc                   809177 non-null  object 
 3   LocationName                808918 non-null  object 
 4   DataSource                  809177 non-null  object 
 5   Category                    809177 non-null  object 
 6   Measure                     809177 non-null  object 
 7   Data_Value_Unit             809177 non-null  object 
 8   Data_Value_Type             809177 non-null  object 
 9   Data_Value                  809177 non-null  float64
 10  Data_Value_Footnote_Symbol  0 non-null       float64
 11  Data_Value_Footnote         0 non-null       float64
 12  Low_Confidence_Limit        780890 non-null  float64
 13  High_Confidence_Limi

We can drop the misspelled `Geolocatioin` after filling in the nas in `Geolocation`. This concept applies to `MeasureID` and `MeasureId` as well.

In [91]:
# Fill NaNs in Geolocation and with Geolocatioin
df_all.loc[df_all['Geolocation'].isna(), 'Geolocation'] = df_all.loc[df_all['Geolocation'].isna(), 'Geolocatioin']

# Apply the same method to `MeasureID
df_all.loc[df_all['MeasureID'].isna(), 'MeasureID'] = df_all.loc[df_all['MeasureID'].isna(), 'MeasureId']

df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 809177 entries, 0 to 28286
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Year                   809177 non-null  object 
 1   StateAbbr              809177 non-null  object 
 2   StateDesc              809177 non-null  object 
 3   LocationName           808918 non-null  object 
 4   DataSource             809177 non-null  object 
 5   Category               809177 non-null  object 
 6   Measure                809177 non-null  object 
 7   Data_Value_Unit        809177 non-null  object 
 8   Data_Value_Type        809177 non-null  object 
 9   Data_Value             809177 non-null  float64
 10  Low_Confidence_Limit   780890 non-null  float64
 11  High_Confidence_Limit  780890 non-null  float64
 12  TotalPopulation        809177 non-null  int64  
 13  CategoryID             809177 non-null  object 
 14  MeasureId              780890 non-null  ob

We can drop nas in any columns we are not already dropping such as `LocationName` and the updated `Geolocation`. `LocationID` refers to the zipcode so we should first see if we can impute with the proper zipcode before droping nas.

> For now, we will also drop any nas from `Low_Confidence_Limit` and `High_Confidence_Limit` and keep the columns as they can be helpful when interpreting our final results when we are evaluating the metrics we will use with our model.

In [92]:
print(df_all.isna().sum()/df_all.shape[0])

df_all.dropna(subset=['LocationName','Geolocation','Low_Confidence_Limit','High_Confidence_Limit'],inplace=True)

Year                     0.000000
StateAbbr                0.000000
StateDesc                0.000000
LocationName             0.000320
DataSource               0.000000
Category                 0.000000
Measure                  0.000000
Data_Value_Unit          0.000000
Data_Value_Type          0.000000
Data_Value               0.000000
Low_Confidence_Limit     0.034958
High_Confidence_Limit    0.034958
TotalPopulation          0.000000
CategoryID               0.000000
DataValueTypeID          0.000000
Short_Question_Text      0.000000
LocationID               0.217515
Geolocation              0.000320
MeasureID                0.000000
dtype: float64


Looks like we were able to fill the nas using the associated `LocationID` with existing `LocationName`.

In [93]:
# Filter rows with non-null LocationID
loc_df = df_all.loc[df_all['LocationID'].notna(), ['LocationName', 'LocationID']]

# Create LocationName <> LocationID dictionary
loc_dict = dict(zip(loc_df['LocationName'], loc_df['LocationID']))

# Fill missing LocationID values based on LocationName from the dictionary
df_all['LocationID'] = df_all['LocationID'].fillna(df_all['LocationName'].map(loc_dict))

# Check the percentage of missing values in df_all after filling missing values
print(df_all.isna().sum() / df_all.shape[0])

Year                     0.0
StateAbbr                0.0
StateDesc                0.0
LocationName             0.0
DataSource               0.0
Category                 0.0
Measure                  0.0
Data_Value_Unit          0.0
Data_Value_Type          0.0
Data_Value               0.0
Low_Confidence_Limit     0.0
High_Confidence_Limit    0.0
TotalPopulation          0.0
CategoryID               0.0
DataValueTypeID          0.0
Short_Question_Text      0.0
LocationID               0.0
Geolocation              0.0
MeasureID                0.0
dtype: float64


### Column Exploration

<ins>**Observations**</ins>

- Looks like we may want to drop the rows with US as the state as it is an error. We can also drop `StateDesc` since it's the same info as `StateAbbr`.
- `DataSource` can be dropped since all of the information is coming from the Behavioral Risk Factor Surveillance System.
- `Data_Value_Unit` can be dropped since all of our data values are in percentages.
- `Data_Value_Type` will be helpful for interpretation of our predictions since it denotes what percentage the `Data_Value` represents. `DataValueTypeID` corresponds to this feature and can be dropped since `Data_Value_Type` is more informative.
- `CategoryID` corresponds to `Category` so we can drop it since `Category is easier to interpret.
- `MeasureId` corresponds the same way to `Measure`. However `Measure` values can be quite lengthy depending on the measure. We also have `Short_Question_Text' corresponding to these features and is more informative than `MeausureId` but shorter than `Measure` so we will keep `Short_Question_Text` and create a reference dictionary before dropping the other columns.

In [101]:
print(df_all['Year'].value_counts(),'\n')
print(df_all['StateAbbr'].value_counts(),'\n')
print(df_all['DataSource'].value_counts(),'\n')
print(df_all['Data_Value_Unit'].value_counts(),'\n')
print(df_all['Data_Value_Type'],'\n')
print(df_all['CategoryID'].value_counts(),'\n')
print(df_all['MeasureID'].value_counts(),'\n')
print(df_all['DataValueTypeID'].value_counts(),'\n')
print(df_all['Short_Question_Text'].value_counts(),'\n')

Year
2020    213724
2018    201088
2021    178408
2019    162284
2017     25136
Name: count, dtype: int64 

StateAbbr
TX    63500
GA    39750
VA    33250
KY    30000
MO    28750
KS    26250
IL    25500
NC    25000
IA    24750
TN    23750
NE    23250
IN    23000
OH    22000
MN    21750
MI    20750
MS    20500
OK    19250
AR    18750
WI    18000
PA    16750
AL    16750
SD    16500
LA    16000
CO    16000
NY    15500
CA    14500
MT    14000
WV    13750
ND    13250
FL    12864
SC    11500
ID    11000
WA     9750
OR     9000
NM     8250
AK     7368
UT     7250
MD     6000
WY     5750
NV     4250
NJ     4158
ME     4000
AZ     3750
VT     3500
MA     3500
NH     2500
CT     2000
RI     1250
HI     1250
DE      750
DC      250
Name: count, dtype: int64 



KeyError: 'DataSource'

In [99]:
# Create reference dictionary for category and measure
measure_df = df_all[['Measure','Short_Question_Text','MeasureID']]
measure_dict = dict(zip(measure_df['MeasureID'], zip(measure_df['Short_Question_Text'], measure_df['Measure'])))
category_df = df_all[['Category','CategoryID']]
category_dict = dict(zip(category_df['Category'],category_df['CategoryID']))

In [100]:
# Drop rows with US as state
df_all = df_all.loc[df_all['StateAbbr'] != 'US']

# Drop other relevant columns
df_all.drop(columns=['DataSource','Data_Value_Unit', 'Low_Confidence_Limit',
                     'High_Confidence_Limit','CategoryID','DataValueTypeID','StateDesc'],inplace=True)
df_all.columns

Index(['Year', 'StateAbbr', 'LocationName', 'Category', 'Measure',
       'Data_Value_Type', 'Data_Value', 'TotalPopulation',
       'Short_Question_Text', 'LocationID', 'Geolocation', 'MeasureID'],
      dtype='object')

Let's quickly check for duplicates since we had to download all of the PLACES datasets separately to accomodate the year range in the SDOH dataset.

In [17]:
dups = df_all.duplicated()
dups.value_counts()

False    809129
True         48
Name: count, dtype: int64

In [18]:
df_all.drop_duplicates(ignore_index=True, inplace=True)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809129 entries, 0 to 809128
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Year                        809129 non-null  object 
 1   StateAbbr                   809129 non-null  object 
 2   StateDesc                   809129 non-null  object 
 3   LocationName                808870 non-null  object 
 4   DataSource                  809129 non-null  object 
 5   Category                    809129 non-null  object 
 6   Measure                     809129 non-null  object 
 7   Data_Value_Unit             809129 non-null  object 
 8   Data_Value_Type             809129 non-null  object 
 9   Data_Value                  809129 non-null  float64
 10  Data_Value_Footnote_Symbol  0 non-null       float64
 11  Data_Value_Footnote         0 non-null       float64
 12  Low_Confidence_Limit        780842 non-null  float64
 13  High_Confidenc

In [56]:
# Final PLACES df
df_all.info()

# Save df_all as csv
df_all.to_csv('df_all_clean.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 780640 entries, 49 to 228769
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Year                 780640 non-null  int64  
 1   StateAbbr            780640 non-null  object 
 2   LocationName         780640 non-null  object 
 3   Category             780640 non-null  object 
 4   Measure              780640 non-null  object 
 5   Data_Value_Type      780640 non-null  object 
 6   Data_Value           780640 non-null  float64
 7   TotalPopulation      780640 non-null  int64  
 8   MeasureId            780640 non-null  object 
 9   Short_Question_Text  780640 non-null  object 
 10  LocationID           780640 non-null  float64
 11  Geolocation          780640 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 77.4+ MB


In [31]:
df_all.loc[:,('Measure','TotalPopulation','LocationName')]

,Measure,TotalPopulation,LocationName
0,Current lack of health insurance among adults ...,327167434,NaN
1,Current lack of health insurance among adults ...,327167434,NaN
2,Arthritis among adults aged >=18 years,327167434,NaN
3,Arthritis among adults aged >=18 years,327167434,NaN
4,Binge drinking among adults aged >=18 years,327167434,NaN
...,...,...,...
28282,No broadband internet subscription among house...,7055,Swisher County
28283,No high school diploma among adults aged 25 ye...,11465,Greensville County
28284,No high school diploma among adults aged 25 ye...,7568,Clay County
28285,Housing cost burden among households,8494,Jack County


In [33]:
sdoh['LocationName']
df_all['LocationName']

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
            ...    
228765        Brown
228766      Grayson
228767     Richland
228768     Franklin
228769    La Crosse
Name: LocationName, Length: 780890, dtype: object

## 3. Data Preparation

In [ ]:
# Drop unneeded columns

